In [1]:
import pandas as pd
from fastapi.responses import JSONResponse

In [2]:
df_gfec=pd.read_parquet("data/Generos_fecha.parquet")

df_ustiempo=pd.read_parquet('data/Usuarios_tiempo.parquet')
df_ur=pd.read_parquet('data/func_3.parquet') 

In [3]:
def PlayTimeGenre( genero ): 
    #Debe devolver año con mas horas jugadas para dicho género.
    #Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}
    if not isinstance(genero, str):
        print("Genero ingresado incorrectamente")
        return
    
    generos=df_gfec[df_gfec["genres"]==genero ]
    if generos.empty:
        return {'Ese genero no existe'}
         
    # Agrupar por "release_date" y sumar la columna "time_forever"
    resultados_agrupados = generos.groupby('anio')['playtime_forever'].sum()
    # Agrupar por "release_date" y sumar la columna "time_forever"
    
    # Encontrar la fecha con el valor máximo
    max_release_date = resultados_agrupados.idxmax()
    # Mostrar el resultado
    print(f"Año de lanzamiento con más horas jugadas para {genero}: {max_release_date}")

   
    return

In [6]:
PlayTimeGenre("Indie")

Año de lanzamiento con más horas jugadas para Indie: 2006


In [7]:
def UserForGenre(genero: str):
    #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación
    # de horas jugadas por año.
    #Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
    # "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
    # Comprobar si el género es una cadena
    if not isinstance(genero, str):
        return {"Error": "Género ingresado incorrectamente"}

    # Filtrar el DataFrame por el género dado
    generos = df_ustiempo[df_ustiempo["genres"] == genero]

    # Comprobar si el género existe en el DataFrame
    if generos.empty:
        return {'Error': 'Ese género no existe'}

    # Agrupar por "user_id" y sumar la columna "playtime_forever"
    resultados_agrupados = generos.groupby('user_id')['playtime_forever'].sum()

    # Encontrar el usuario con más horas jugadas
    max_por_id = resultados_agrupados.idxmax()

    # Filtrar el DataFrame para contener solo las filas con 'user_id' igual al máximo
    df_max = generos[generos['user_id'] == max_por_id]

    # Crear una lista de acumulación de horas jugadas por año
    acumulacion_horas_anio = df_max.groupby(df_max['anio'])['playtime_forever'].sum()

    # Crear una lista de diccionarios con el año y las horas jugadas
    lista_acumulacion = [{"Año": anio, "Horas": horas} for anio, horas in acumulacion_horas_anio.items()]

    respuesta = {
        "Usuario con más horas jugadas para {}".format(genero): max_por_id,
        "Horas jugadas": lista_acumulacion
    }

    return respuesta


In [8]:
UserForGenre( "RPG" )

{'Usuario con más horas jugadas para RPG': 'Icebrrg1245',
 'Horas jugadas': [{'Año': '2011', 'Horas': 49013},
  {'Año': '2013', 'Horas': 127},
  {'Año': '2014', 'Horas': 300977}]}

In [9]:
def UsersRecommend(anio: int):
    positivos = df_ur[(df_ur['anio'] == anio) & (df_ur['recommend'] == True) & (df_ur['sentiment_analysis'] >= 0)]

    if positivos.empty:
        return {"No hay datos para ese año"}

    recomendaciones = positivos['item_id'].value_counts().reset_index()
    recomendaciones.columns = ['item_id', 'count']
    
    recomendaciones = pd.merge(recomendaciones, df_ur[['item_id', 'item_name']], on='item_id', how='left')
    recomendaciones = recomendaciones.drop_duplicates()
    
    top_3_games = recomendaciones.sort_values(by='count', ascending=False).head(3)
    top_3_games = top_3_games.reset_index(drop=True)
    print(top_3_games)
    respuesta = [{"Puesto " + str(i + 1): game['item_name']} for i, game in top_3_games.iterrows()]

    return JSONResponse(content=respuesta)


In [142]:
UsersRecommend( 2012)

   item_id  count      item_name
0   105600     43       Terraria
1     4000     39    Garry's Mod
2      550     36  Left 4 Dead 2


In [11]:
def UsersNotRecommend( año : int ):
    #Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.
    # (reviews.recommend = False y comentarios negativos)
    positivos = df_ur[(df_ur['anio'] == año) & (df_ur['recommend'] == False) ]

    # Contar cuántas revisiones tiene cada juego
    recomendaciones = positivos['item_id'].value_counts().reset_index()
    recomendaciones.columns = ['item_id', 'count']

    # Fusionar 'recomendaciones' con 'df_ur' para obtener los nombres
    recomendaciones = pd.merge(recomendaciones, df_ur[['item_id', 'item_name']], on='item_id', how='left')
    # Realizo el merge de ambos DataFrames borro duplicados
    recomendaciones=recomendaciones.drop_duplicates()

    # Ordenar los juegos en función del número de revisiones en orden descendente
    game_recommendations = recomendaciones.sort_values(by='count', ascending=False)

    # Tomar los 3 juegos más recomendados
    top_3_games = game_recommendations.head(3)
    #merged_df = pd.merge(top_3_games, df_ur, left_on='item_id', right_on='item_id', how='inner')
    # Tomar los 3 juegos más recomendados
    top_3_games = game_recommendations.head(3)

    # Crear el formato de salida
    top_3_games = [{"Puesto " + str(i + 1): game} for i, game in enumerate(top_3_games['item_name'])]
    if top_3_games:
        print(top_3_games)
    else:
        print("No hay datos para esa fecha")
    return

In [12]:
UsersNotRecommend(2011)

[{'Puesto 1': 'And Yet It Moves'}, {'Puesto 2': 'Fable III'}, {'Puesto 3': 'Counter-Strike: Source'}]


In [13]:
def sentiment_analysis( año : int ):
    #Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios
    # que se encuentren categorizados con un análisis de sentimiento.
    #Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}
    # Agrupar por año y contar las categorías de análisis de sentimiento
    
    sent_por_anio = df_ur.groupby('anio')['sentiment_analysis'].value_counts().unstack(fill_value=0)
   
    # Verificar si el año existe en el índice del DataFrame
    if año in sent_por_anio.index:
        # El año existe, así que puedo filtrar 
        sent_por_anio = sent_por_anio.loc[año].to_dict()
          
        print(f"Negativo : {sent_por_anio[0]} , Neutral : {sent_por_anio[1]} Positivo: {sent_por_anio[2]}")
    else:
        print(f"No se encontraron datos para el año {año}.")
    return


In [14]:
sentiment_analysis( 2013)

Negativo : 855 , Neutral : 471 Positivo: 4265
